In [1]:
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics import classification_report
import numpy as np

# 🚀 Cihaz seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📦 Kullanılan cihaz: {device}")

# 📂 Klasör yolları
train_fake_dir = r"C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\train\train-fake-frame"
train_real_dir = r"C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\train\train-real-frame"
test_fake_dir = r"C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\test\test-fake-frame"
test_real_dir = r"C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\test\test-real-frame"

# 🔧 Ayarlar
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

# 🎨 Görüntü transformasyonu
print("🎨 Transform işlemleri ayarlanıyor...")
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 🧰 Custom Dataset Sınıfı
class FrameDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        print(f"📥 Dataset oluşturuluyor: \n   Real: {real_dir}\n   Fake: {fake_dir}")
        self.real_images = [os.path.join(real_dir, img) for img in os.listdir(real_dir) if img.lower().endswith(('.jpg', '.png'))]
        self.fake_images = [os.path.join(fake_dir, img) for img in os.listdir(fake_dir) if img.lower().endswith(('.jpg', '.png'))]
        self.transform = transform
        self.samples = [(img_path, 0) for img_path in self.real_images] + \
                       [(img_path, 1) for img_path in self.fake_images]
        print(f"🧮 Toplam görüntü sayısı: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# 📦 Dataset ve Dataloader
print("📦 Eğitim ve test dataset'leri oluşturuluyor...")
train_dataset = FrameDataset(train_real_dir, train_fake_dir, transform=transform)
test_dataset = FrameDataset(test_real_dir, test_fake_dir, transform=transform)

print("📦 Dataloader'lar başlatılıyor...")
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 🧠 Model: MobileNetV2 (Binary classifier)
print("🧠 MobileNetV2 modeli hazırlanıyor...")
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# -------------------------------
# ⏱ Eğitim
# -------------------------------
print("\n🚀 Eğitim başlıyor...")
train_start = time.time()
model.train()
for epoch in range(EPOCHS):
    print(f"\n🎯 Epoch {epoch+1}/{EPOCHS}")
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i+1) % 100 == 0:
            print(f"   🔁 Batch {i+1}/{len(train_loader)} - Loss: {loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} tamamlandı - Ortalama Loss: {avg_loss:.4f}")
train_end = time.time()

# 💾 Modeli Kaydet
model_path = "mobilenetv2_deepfake.pth"
torch.save(model.state_dict(), model_path)
print(f"\n💾 Model kaydedildi: {model_path}")

# -------------------------------
# ⏱ Test
# -------------------------------
print("\n🔬 Test başlıyor...")
test_start = time.time()
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
        all_preds.extend(preds.astype(int).flatten())
        all_labels.extend(labels.numpy())

        if (i+1) % 50 == 0:
            print(f"   🔎 Test batch {i+1}/{len(test_loader)} işlendi...")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["real", "fake"]))
test_end = time.time()

# -------------------------------
# ⏱ Süre Bilgisi
# -------------------------------
print(f"\n⏱ Eğitim süresi: {train_end - train_start:.2f} saniye")
print(f"⏱ Test süresi: {test_end - test_start:.2f} saniye")


📦 Kullanılan cihaz: cuda
🎨 Transform işlemleri ayarlanıyor...
📦 Eğitim ve test dataset'leri oluşturuluyor...
📥 Dataset oluşturuluyor: 
   Real: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\train\train-real-frame
   Fake: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\train\train-fake-frame
🧮 Toplam görüntü sayısı: 270634
📥 Dataset oluşturuluyor: 
   Real: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\test\test-real-frame
   Fake: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\test\test-fake-frame
🧮 Toplam görüntü sayısı: 67671
📦 Dataloader'lar başlatılıyor...
🧠 MobileNetV2 modeli hazırlanıyor...


C:\Users\Admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🚀 Eğitim başlıyor...

🎯 Epoch 1/20
   🔁 Batch 100/8458 - Loss: 0.4932
   🔁 Batch 200/8458 - Loss: 0.2878
   🔁 Batch 300/8458 - Loss: 0.3237
   🔁 Batch 400/8458 - Loss: 0.2361
   🔁 Batch 500/8458 - Loss: 0.1505
   🔁 Batch 600/8458 - Loss: 0.1132
   🔁 Batch 700/8458 - Loss: 0.1357
   🔁 Batch 800/8458 - Loss: 0.0509
   🔁 Batch 900/8458 - Loss: 0.2109
   🔁 Batch 1000/8458 - Loss: 0.1614
   🔁 Batch 1100/8458 - Loss: 0.1324
   🔁 Batch 1200/8458 - Loss: 0.1480
   🔁 Batch 1300/8458 - Loss: 0.4722
   🔁 Batch 1400/8458 - Loss: 0.0330
   🔁 Batch 1500/8458 - Loss: 0.1094
   🔁 Batch 1600/8458 - Loss: 0.1133
   🔁 Batch 1700/8458 - Loss: 0.2042
   🔁 Batch 1800/8458 - Loss: 0.1736
   🔁 Batch 1900/8458 - Loss: 0.0643
   🔁 Batch 2000/8458 - Loss: 0.0679
   🔁 Batch 2100/8458 - Loss: 0.1144
   🔁 Batch 2200/8458 - Loss: 0.0797
   🔁 Batch 2300/8458 - Loss: 0.0735
   🔁 Batch 2400/8458 - Loss: 0.0719
   🔁 Batch 2500/8458 - Loss: 0.1534
   🔁 Batch 2600/8458 - Loss: 0.1255
   🔁 Batch 2700/8458 - Loss: 0.1045
 